# Data exploration #1

### In this notebook we will perform a preliminary statistical exploration of the dataset in order to get a better sense of the problem's formulation and any outliers or extreme scenarios that we should take care.

In [4]:
import tensorflow as tf
import datetime
import numpy as np
import pandas as pd
import os
import sys
from scipy import stats
import plotly.plotly as py
from plotly import tools
import numpy as np
import cufflinks
import plotly.figure_factory as ff
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
import numpy as np
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
CSV_PATH = module_path + "/data/raw/routes.csv"


dataset = pd.read_csv(
    CSV_PATH, sep='	', low_memory=False)

In [6]:
# Quick peek at the dataset
dataset

,passenger_id,source_latitude,source_longitude,source_address,destination_latitude,destination_longitude,destination_address,request_date
0,41037,-12.088156,-77.016065,"Avenida Nicolás de Arriola 314, La Victoria 13",-12.108531,-77.044891,"Calle Carlos Graña Elisande 340, San Isidro 27",2015-09-01 00:00:04
1,116591,-12.099957,-77.036497,"Av Los Conquistadores 392, San Isidro 15073",-12.119686,-76.999969,"Bruselas 228, La Calera De La Merced",2015-09-01 00:00:15
2,86426,-12.099153,-77.019425,"Av. República de Panamá 3537, San Isidro 27",-12.076505,-77.089305,"Av. La Marina cdra. 25, San Miguel 32",2015-09-01 00:00:17
3,53610,-12.110271,-77.028945,"Junín 225, Miraflores",-12.132221,-77.027021,"Calle San Fernando 380, Miraflores 18",2015-09-01 00:00:29
4,102927,-12.098430,-77.026246,"Av. República De Colombia 791, San Isidro",-12.099529,-76.990486,"Calle Mozart 201, San Borja 41",2015-09-01 00:00:31
5,163806,-12.142475,-77.011614,"Jr. Rivera, Surco",-12.083100,-77.052130,"Avenida San Felipe, Jesús María 11",2015-09-01 00:00:34
6,96702,-12.106040,-77.038510,Pardo Y Aliaga 652,-12.092780,-77.062653,"Avenida Javier Prado Oeste 654, Magdalena del ...",2015-09-01 00:00:50
7,173422,-12.088830,-76.989520,"Cavallini 261, Lima 15037",-12.133783,-77.010262,"Carlos Baca Flor 646, Lima 15048",2015-09-01 00:00:54
8,116591,-12.099957,-77.036497,"Av Los Conquistadores 392, San Isidro 15073",-12.119686,-76.999969,"Bruselas 228, La Calera De La Merced",2015-09-01 00:00:57
9,226380,-12.076007,-77.037887,"Avenida Francisco Javier Mariategui 183, Jesús...",-12.089933,-77.052312,"Av. Salaverry 2370, Jesús María 00011",2015-09-01 00:00:58


### Next, lets extract some series of variable granularity to get a sense of the std and the mean of the volume per week, day, hour and 15 mins.

### "Quick and Dirty" Aggregation:

In [12]:
from src.processing import dataProc

dataset['ride_volume'] = 1

quart_sum = dataProc.aggregate_by_mins(
    dataset=dataset, datetime_column='request_date', minutes=15)

hour_sum = dataProc.aggregate_by_hours(
    dataset=dataset, datetime_column='request_date', hours=1)

day_sum = dataProc.aggregate_by_hours(
    dataset=dataset, datetime_column='request_date', hours=24)

week_sum = dataProc.aggregate_by_hours(
    dataset=dataset, datetime_column='request_date', hours=24 * 7)

900


# Now that we have the sums for a gradient granularity, we can start checking for outliers:

In [13]:
def plot_outlier_boxes(data_array):
    # Create distplot with custom bin_size
    txt = hour_sum.index.tolist()

    trace0 = go.Box(
        y=data_array['ride_volume'],
        name="All Points",
        jitter=0.3,
        pointpos=-1.8,
        boxpoints='all',
        text=txt,
        marker=dict(
            color='rgb(7,40,89)'),
        line=dict(
            color='rgb(7,40,89)')
    )

    trace1 = go.Box(
        y=data_array['ride_volume'],
        text=txt,
        name="Only Whiskers",
        boxpoints=False,
        marker=dict(
            color='rgb(9,56,125)'),
        line=dict(
            color='rgb(9,56,125)')
    )

    trace2 = go.Box(
        y=data_array['ride_volume'],
        text=txt,
        name="Suspected Outliers",
        boxpoints='suspectedoutliers',
        marker=dict(
            color='rgb(8,81,156)',
            outliercolor='rgba(219, 64, 82, 0.6)',
            line=dict(
                outliercolor='rgba(219, 64, 82, 0.6)',
                outlierwidth=2)),
        line=dict(
            color='rgb(8,81,156)')
    )

    trace3 = go.Box(
        y=data_array['ride_volume'],
        text=txt,
        name="Whiskers and Outliers",
        boxpoints='outliers',
        marker=dict(
            color='rgb(107,174,214)'),
        line=dict(
            color='rgb(107,174,214)')
    )

    data = [trace0, trace1, trace2, trace3]

    layout = go.Layout(
        title="Box Plot Styling Outliers"
    )

    fig = go.Figure(data=data, layout=layout)
    return fig


# Checking per hour outliers
fig = plot_outlier_boxes(hour_sum)
iplot(fig, filename="Box Plot Styling Outliers")

In [14]:
# Plotting the histogram per hour for verification
data = [go.Histogram(x=hour_sum['ride_volume'])]
iplot(data, filename='Day histogram')

## From the graphs above it is clear that the aggregation produces outliers that must be removed. We are going to use the z score to do that:

In [16]:
threshold = 3
hour_list = hour_sum['ride_volume']
z = np.abs(stats.zscore(hour_list))
outliers = list(np.where(z > threshold)[0])
# for index in sorted(outliers, reverse=True):
#     hour_sum.drop(hour_sum.index[index])

    
hour_sum = hour_sum.drop(hour_sum.index[outliers])

In [17]:
# Checking per hour outliers
fig = plot_outlier_boxes(hour_sum)
iplot(fig, filename="Box Plot Styling Outliers")


### Outliers can be trimmed in order to achieve a greater prediction accurasy but in our case our data is seasonal, and many outliers are due to that seasonality. 
### Also, trimming a time series dataset can produce time gaps which may be more detrimental to the model's training.
# Hence, we will be working with the untrimmed dataset as is for our predictions.